# Lab 5: Framing

## 1. Install Semantic Link Labs Python Library
This step installs Semantic Link Library which is a Python library design for use in Microsoft Fabric Notebooks.  The library extends the capabilities of [Semantic Link](https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview) offering additional functionalities to seamlessly integrate alongside it.

In [ ]:
%pip install -q --disable-pip-version-check semantic-link-labs==0.10.1

## 2. Install Python Libraries
This step does the following:
- Sets up libraries that will be used iater in the script for various functions related to data processing, manipulation and handling
- Creates a populates the following variables
    - LakehouseName - Used as the name for the Lakehouse that will be created later in this script
    - SemanticModelName = Used as the  name for the Semantic Model that will be created later in this script

In [ ]:
import sempy_labs as labs
import time
import pandas

LakehouseName = "AdventureWorks"
lakehouses = labs.list_lakehouses()["Lakehouse Name"]
for l in lakehouses:
    if l.startswith("Adventure"):
        LakehouseName = l

SemanticModelName = f"{LakehouseName}_model"
workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]

## 4. Load libraries to help generate a Power BI report

In [ ]:
from sempy_labs._helper_functions import (
    resolve_report_id,
    format_dax_object_name,
    resolve_dataset_from_report,
    _conv_b64,
    _extract_json,
    _add_part,
    _decode_b64,
    resolve_workspace_name_and_id,
    _update_dataframe_datatypes,
    _base_api,
    _create_dataframe,
)
from typing import Optional, List
import pandas as pd
import json
import base64
from uuid import UUID
from sempy._utils._log import log
import sempy_labs._icons as icons
import sempy_labs.report._report_helper as helper
from sempy_labs._model_dependencies import get_measure_dependencies
from jsonpath_ng.ext import parse
import warnings
import requests

## 5. Create Simple Power BI Report to show data modification changes 

In [ ]:
report_name="Simple Report"

pbi_report:dict = {}
pbi_report['config'] = """{
        "version": "5.37",
        "themeCollection": {},
        "activeSectionIndex": 0,
        "linguisticSchemaSyncVersion": 0,
        "objects": {
            "outspacePane": [
                {
                    "properties": {
                        "expanded": {
                            "expr": {
                                "Literal": {
                                    "Value": "false"
                                }
                            }
                        }
                    }
                }
            ]
        }
    }"""
pbi_report['layoutOptimization']=0
pbi_report['resourcePackages'] = [{'resourcePackage': {'disabled': False, 'items': [{'name': 'CY24SU10', 'path': 'BaseThemes/CY24SU10.json', 'type': 202}], 'name': 'SharedResources', 'type': 2}}]
pbi_report['sections'] = [
    {'config': '{}', 
    'displayName': 'Page 1', 
    'displayOption': 1, 
    'filters': '[]', 
    'height': 300.0, 
    'width': 600.0,
    'name': 'a4c1ed461808909ae820', 
    'visualContainers':
        [
            {'config': '''{
                        "name": "Matrix",
                        "layouts": [
                            {
                                "id": 0,
                                "position": {
                                    "x": 310,
                                    "y": 30,
                                    "z": 1000,
                                    "width": 253,
                                    "height": 202
                                }
                            }
                        ],
                        "singleVisual": {
                            "visualType": "tableEx",
                            "projections": {
                                "Values": [
                                    {
                                        "queryRef": "DimDate.Month"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.SalesAmount)"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.DiscountAmount)"
                                    }
                                ]
                            },
                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "d",
                                        "Entity": "DimDate",
                                        "Type": 0
                                    },
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "d"
                                                }
                                            },
                                            "Property": "Month"
                                        },
                                        "Name": "DimDate.Month",
                                        "NativeReferenceName": "Month"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "SalesAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.SalesAmount)",
                                        "NativeReferenceName": "Sum of SalesAmount"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "DiscountAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.DiscountAmount)",
                                        "NativeReferenceName": "Discount"
                                    }
                                ]
                            },
                            "columnProperties": {
                                "Sum(FactInternetSales.DiscountAmount)": {
                                    "displayName": "Discount"
                                }
                            },
                            "drillFilterOtherVisuals": true,
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    }''', 'filters': '[]', 'height': 202.46, 'width': 215.11, 'x': 319.67, 'y': 30.63, 'z': 1.0
                },
            {'config': 
                    '''{
                        "name":"Card",
                        "layouts":[
                            {
                            "id":0,
                            "position":{"x":10,"y":30,"z":0,"width":238,"height":201}}
                            ],
                        "singleVisual":{"visualType":"card","projections":{"Values":[{"queryRef":"FactInternetSales.Count of Sales"}]},


                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Measure": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "f"
                                                }
                                            },
                                            "Property": "Count of Sales"
                                        },
                                        "Name": "FactInternetSales.Count of Sales",
                                        "NativeReferenceName": "Count of Sales"
                                    }
                                ],
                                "OrderBy": [
                                    {
                                        "Direction": 2,
                                        "Expression": {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {
                                                        "Source": "f"
                                                    }
                                                },
                                                "Property": "Count of Sales"
                                            }
                                        }
                                    }
                                ]
                            },



                            "drillFilterOtherVisuals":true,
                            "hasDefaultSort":true,                           
                            "objects": {
                                "labels": [
                                    {
                                        "properties": {
                                            "fontSize": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "20D"
                                                    }
                                                }
                                            },
                                            "labelDisplayUnits": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "1D"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            },
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                            }
                        }''',
                 'filters': '[]', 
                 'height': 201.5, 
                 'width': 265.43, 
                 'x': 270.03, 
                 'y': 30.12, 
                 'z': 1000.0
            }
        ]
    }]

labs.report.create_report_from_reportjson(report=report_name , dataset="AdventureWorks_model" , report_json = pbi_report)
report_id = resolve_report_id(report_name)

from powerbiclient import Report
report = Report(group_id=None, report_id=report_id)
report.set_size(400,700)
report

## 6. Show Lakehouse Tables

In [ ]:
labs.lakehouse.get_lakehouse_tables(LakehouseName)

## 7. Show relationships for AdventureWorks_model

In [ ]:
import sempy.fabric as fabric
from sempy.relationships import plot_relationship_metadata , find_relationships

relationships = fabric.list_relationships(SemanticModelName)
plot_relationship_metadata(relationships)


## 8. Show history details for **DimDate**
Look at small table first

In [ ]:
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="DimDate"))

## 9. Show history details for **FactInternetSales**

In [ ]:
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 10. Insert data to **FactInternetSales** using Append
load an existing day of data to a dataframe
update the OrderDateKey for all rows in the dataframe to 20221204
Append the rows in the dataframe back to the FactInternetSales Delta Table

In [ ]:
# Get one day of data from existing table
from pyspark.sql.functions import lit, min, max ,count
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
# df1.agg(
#     min("OrderDateKey").alias("min_OrderDateKey") ,
#     max("OrderDateKey").alias("max_OrderDateKey") ,
#     count("*").alias("count_rows")
#     ).show()


# Create a filtered dataframe to update and then append back onto the original table
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050630))


df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

In [ ]:
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
df1.agg(
        min("OrderDateKey").alias("min_OrderDateKey") ,
        max("OrderDateKey").alias("max_OrderDateKey") ,
        count("*").alias("count_rows")
        ).show()


## 11. Load **FactInternetSales** into variable
To perform some updates and deletes

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")


## 12. **Delete** some rows from **FactInternetSales**

In [ ]:
deltaTable.delete("OrderDateKey = '20050701'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 13. **Update** some rows in **FactInternetSales**
Set all values for OrderDateKey = 20220218 to be DiscountAmount of 1 (was 0)

In [ ]:
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"1"}
)
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 14. Turn OFF auto framing
Must be done manually on the **AdventureWorks_model**

Make sure you click apply

## 15. Insert _more_ data to **FactInternetSales** using Append

In [ ]:
# Create a filtered dataframe to update and then append back onto the original table
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050629))


df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 16. **Update** some rows from **FactInternetSales**

In [ ]:
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"2"}
)
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 17. **Delete** some rows from **FactInternetSales**

In [ ]:
deltaTable.delete("OrderDateKey = '20050702'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 18. Reframe model to update changes
This code block attempts to reframe the Semantic model in a loop until successful, catching exceptions and retrying every 3 seconds.  Upon, success, it prints a confirmation message.

In [ ]:
reframeOK:bool=False
while not reframeOK:
    try:
        result:pandas.DataFrame = labs.refresh_semantic_model(dataset=SemanticModelName)
        reframeOK=True
    except:
        print('Error with reframe... trying again.')
        triggerMetadataRefresh()
        time.sleep(3)

print('Custom Semantic Model reframe OK')
report

## 19. Stop the Spark session

In [ ]:
mssparkutils.session.stop()